In [4]:
import pandas as pd
import numpy as np
from collections import defaultdict
import re
from collections import Counter
import math

In [5]:
documents = pd.read_csv("Questions.csv",  encoding='latin-1')

In [9]:
## Just used the first 500 questions 
small_documents = documents['Title'].head(500)

In [10]:
small_documents.head()

0    How can I find the full path to a font from it...
1              Get a preview JPEG of a PDF on Windows?
2    Continuous Integration System for a Python Cod...
3       cx_Oracle: How do I iterate over a result set?
4    Using 'in' to match an attribute of Python obj...
Name: Title, dtype: object

In [11]:
## quick coalesce function to prevent division by 0 error

def coalesce(value):
    if value == 0:
        return np.NaN
    else:
        return value

In [14]:
## figure out the unique set of words -- bag of words

all_unique_words = []
for document in small_documents:
    for word in re.sub('[^A-Za-z0-9]+', ' ', document).split():
        if word.lower() not in all_unique_words:
            all_unique_words.append(word.lower())

In [53]:
## figure out frequency for each word in each document 

## denominator
all_words = []
for document in small_documents:
    for word in document.split():
        all_words.append(word.lower())
counts = Counter(all_words)

## calculate TF
tf = defaultdict(list)
for i in all_unique_words:
    for index, document in enumerate(small_documents):
        for x in document.split():
            frequency = 0
            if x.lower() == i.lower():
                frequency += 1
        overall_frequency = counts[i]
        tf[i].append(frequency/coalesce(overall_frequency))

In [54]:
## compute IDF

total_documents = len(small_documents)

documents_with_word = defaultdict(int)

for document in small_documents:
    for word in all_unique_words:
        if word in document.lower().split():
            documents_with_word[word] += 1
            
tf_idf = defaultdict(int)

for word in all_unique_words:
    tf_idf[word] = pd.Series(tf[word]) * math.log(total_documents/coalesce(documents_with_word[word]))

In [55]:
def most_relevant_doc(word):
    ## for a given word return the most relevant document ##
    index_max = np.argmax(tf_idf[word])
    return [small_documents[np.argmax(tf_idf[word])], word, tf_idf[word].max()]
    
    

In [60]:
most_relevant_doc("path")

/Users/shirleyliu/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


['Python reading Oracle path', 'path', 0.9210340371976184]